## Operación del generador de funciones de la marca **Tektronix**

En este tutorial vamos a ver:

1- Cómo utilizar la [librería pyvisa](https://pyvisa.readthedocs.io/en/latest/) de python para comunicarnos con el equipo

2- Cómo configurar el equipo para la generación de señales típicas

3- Cómo hacer un barrido en frecuencias

4- Cómo hacer un barrido en amplitud


**Detalles del equipo**: De la marca Tektronix, en el laboratorio esta disponible el modelo AFG1022 

[Ver manual de programación del AFG1022](https://asignaturas.df.uba.ar/l3-raspa/wp-content/uploads/sites/26/2025/02/AFG1022-Programmer-Manual-EN-1.pdf).

### 1- Cómo utilizar pyvisa para cominicarnos con el equipo

Empezamos importando las librerías que vamos a utilizar:

In [ ]:
import pyvisa                               # para comunicarnos con los equipos
import numpy as np                          # para operar con listas de datos
import time                                 # para realizar pausas entre mediciones

En primer lugar, necesitamos un objeto que nos permita comunicarnos e interactuar con los instrumentos que tenemos conectados vía puerto USB, GPIB, Serial (RS232), Ethernet, etc. Ese objeto es el *resource manager* (administrador de recursos) y la invocamos desde la librería pyvisa:

In [ ]:
rm = pyvisa.ResourceManager()

El resource manager (rm) se utiliza para: (a) detectar dispositivos conectados y listar todos los recursos disponibles y (b) crear interfases para comunicarnos con dispositivos.

Una vez definido el objeto **rm**, podemos listar los instrumentos conectados a nuestra PC: 

In [ ]:
instrumentos_conectados = rm.list_resources()           # lista de instrumentos conectados

print (instrumentos_conectados)

De la lista obtenida en la celda anterior, buscamos el generador Tektronix. Copiamos el nombre y lo pegamos en la variable instrumento, luego procedemos a establecer la comunicación con el equipo:

In [ ]:
instrumento = 'puerto::marca::modelo::NroSerie'         # colocar acá el instrumento asociado al osciloscopio 

gen = rm.open_resource(instrumento)                     # Establezco comunicación con el osciloscopio

gen.query('*IDN?')                                      # pido que se identifique para checkear conexión

El objeto *gen* será nuestra interfase, éste nos permitirá interactuar con el generador de funciones utilizando rutinas estándar. Generalmente, solemos realizar dos tipos de operaciones:

- Enviar un comando al instrumento para configurar parámetros o iniciar procesos sin esperar ninguna respuesta. Esto se realiza mediante rutinas de tipo **write**.
  
- Enviar un comando al instrumento y esperar una respuesta, como por ejemplo, obtener los datos que estoy midiendo o consultar el estado del instrumento. Esto se realiza mediante rutinas de tipo **query**.

El lenguaje que utilizamos para comunicarnos con el equipo está definido en el estándar [**SCPI**](https://en.wikipedia.org/wiki/Standard_Commands_for_Programmable_Instruments) (Standard Commands for Programmable Instruments), el cual es ampliamente utilizado para la comunicación remota con equipos de instrumentación como osciloscopios, multímetros y generadores de funciones.

### 2- Generación de funciones

Establecida la conexión con el generador de función, a continuación mostramos como configurar el equipo para que entregue distintos tipos de señales.

#### 2.1. Señal sinusoidal 

En primer lugar, apago la salida, esto es opcional pero es una buena costumbre:

In [ ]:
gen.write("OUTPUT1 OFF")  

Luego indicamos que en el canal 1 se quiere generar una función seno:

In [ ]:
gen.write("SOURCE1:FUNC SIN")

Para generar una función sinusoide necesitamos definir tres parámetros: Amplitud, frecuencia y fase, esto dependerá del experimento. Por ejemplo, supongamos:

In [ ]:
ampli= 4           # voltaje pico a pico
freq = 440         # en Hz 
fase = 180         # en grados

Configuramos la señal con esos parámetros:

In [ ]:
gen.write("SOURCE1:VOLT %f Vpp" %ampli)     # setea la amplitud, le especificamos la unidad Vpp

gen.write("SOURCE1:FREQ %f" %freq)          # setea la frecuencia en Hz

gen.write("SOURCE1:PHASE %f DEG" %fase)     # setea la fase, hay que agregar la unidad (DEG) para usar grados, sino por defecto toma radianes

Listo, la señal quedo configurada ahora encendemos el canal:

In [ ]:
gen.write("OUTPUT1 ON")

#### 2.2. Señal tipo Rampa

Queremos una señal tipo rampa o diente de sierra. Este tipo de señal crece linealmente por un tiempo para luego decaer linealmente.

En forma similar, seteamos al canal 1 como source y le indicamos que genera una función tipo *RAMP*:

In [ ]:
gen.write("OUTPUT1 OFF")  # primero apago la salida

gen.write("SOURCE1:FUNC RAMP")

Con los mismos comandos que antes podemos setear amplitud, frecuencia y fase. A este tipo de función se le agrega el parámetro simetría. 

Por ejemplo queremos setear una rampa de amplitud *4 Vpp* de frecuencia *440 Hz*

In [ ]:
gen.write('SOURCE1:VOLT 4 Vpp') # Seteo de la amplitud pico a pico

gen.write("SOURCE1:FREQ 440" )

gen.write("OUTPUT1 ON") # Una vez seteada, enciendo el canal

### 3. Barrido en frecuencias

Vamos a hacer un barrido en frecuencias de una señal sinusoidal, la seteamos en el generador:

In [ ]:
gen.write("SOURCE1:FUNC SIN")

En primer lugar definimos las frecuencias mínima y máxima de nuestro barrido:

In [ ]:
frec1 = 10.0                # freq mínima
frec2 = 1000.0              # freq máxima

Primero, exploramos un barrido en frecuencias lineal. En este caso, variamos la frecuencia tal que esta se incrementa de manera uniforme entre un valor inicial y un valor final. 

In [ ]:
paso= 50                                           # espaciamiento entre freqs

N1 = int( (frec2-frec1)/paso ) +1                  # cant de freqs en el conjunto

frecuencias = np.linspace( frec1, frec2, N1 )      # conjunto de freqs

for freq in frecuencias:
    print( freq )
    gen.write('SOURCE1:FREQ %f' % freq)            # fija la freq en el gen
    time.sleep(2)

Ahora planteamos un barrido logarítmico. En este caso, las frecuencias de prueba se distribuyen de manera que la relación entre frecuencias consecutivas es constante, en lugar de la diferencia absoluta. Esto permite cubrir un rango amplio de frecuencias con mayor detalle en bajas frecuencias y menos puntos en altas frecuencias, siendo útil en análisis de respuesta en frecuencia y caracterización de sistemas que siguen comportamientos exponenciales o de amplio espectro.

In [ ]:
frec1 = 10.0                # freq mínima
frec2 = 1000.0              # freq máxima

N2=100

frecuencias=np.geomspace( frec1, frec2, N2 )

for freq in frecuencias:
    print(freq)
    gen.write('SOURCE1:FREQ %f' % freq)             # fija freq
    time.sleep(2)    

### 4- Barrido en amplitud